In [0]:
CATALOGO = "v_credit"
SCHEMA_ORIGEM = "gold"
SCHEMA_DESTINO = "curated"
NOME_VIEW = "vw_jornada_cliente"

full_view_name = f"{CATALOGO}.{SCHEMA_DESTINO}.{NOME_VIEW}"

In [0]:
query = f"""
CREATE OR REPLACE VIEW {full_view_name} AS
SELECT
    cliente.ds_faixa_etaria, -- Geração Z vs Senior
    cliente.nm_regiao,
    canal.ds_tipo_interacao, -- Automatizado vs Humanizado
    
    -- Métricas de Experiência
    COUNT(fato.pk_fato_atendimento) AS volume,
    AVG(fato.val_nota_csat) AS csat_medio,
    
    -- Quantos tiveram experiência ruim?
    SUM(cast(fato.fl_experiencia_negativa as double)) AS qtd_clientes_irritados,
    
    -- % de Irritação
    (SUM(cast(fato.fl_experiencia_negativa as double)) / COUNT(*)) * 100 AS perc_experiencia_negativa

FROM {CATALOGO}.{SCHEMA_ORIGEM}.ft_atendimento_geral AS fato
JOIN {CATALOGO}.{SCHEMA_ORIGEM}.dm_cliente AS cliente ON fato.cd_cliente = cliente.cd_cliente
JOIN {CATALOGO}.{SCHEMA_ORIGEM}.dm_canal AS canal ON fato.cd_canal = canal.cd_canal

GROUP BY 1, 2, 3;
"""

spark.sql(query)
print(f"✅ View {full_view_name} criada/atualizada com sucesso!")

In [0]:
print("--- CONTAGEM DE LINHAS ---")
print("Fato:", spark.read.table("v_credit.gold.ft_atendimento_geral").count())
print("Dim Cliente:", spark.read.table("v_credit.gold.dm_cliente").count())
print("Dim Canal:", spark.read.table("v_credit.gold.dm_canal").count())

# 2. Checar se as chaves batem (Amostragem)
print("\n--- AMOSTRA DE CHAVES ---")
print("Chave Fato (cd_cliente):")
spark.sql("SELECT DISTINCT cd_cliente FROM v_credit.gold.ft_atendimento_geral LIMIT 5").show()

print("Chave Dimensão (cd_cliente):")
spark.sql("SELECT DISTINCT cd_cliente FROM v_credit.gold.dm_cliente LIMIT 5").show()